# 🏛️ SmartFreeGov - RAG FAQ Demo

BigQuery + GPT-4 による行政FAQ自動応答デモです。

In [ ]:
# ✅ 必要ライブラリのインストール（最初の1回だけ）
!pip install openai google-cloud-bigquery python-dotenv

In [ ]:
import os

# APIキーは手入力（セキュア）
OPENAI_API_KEY = input("🔑 OpenAI APIキーを入力してください（表示されます）: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# プロジェクト情報はユーザーに任せる
os.environ['BQ_PROJECT_ID'] = input("🧠 BigQueryプロジェクトIDを入力してください: ")
os.environ['BQ_DATASET_ID'] = input("📦 データセットIDを入力してください: ")
os.environ['BQ_TABLE_NAME'] = input("📄 テーブル名を入力してください: ")

In [ ]:
# ✅ GPT呼び出し関数（v1対応）
from openai import OpenAI

def call_chatgpt(prompt):
    client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "あなたは行政FAQのアシスタントです。"},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )
    return response.choices[0].message.content.strip()

In [ ]:
# 🔰 手元で作成したサービスアカウントJSONをアップロード
from google.colab import files
uploaded = files.upload()  # ambient-empire-xyz.json をアップ

# 🔑 認証ファイルを指定してBigQueryクライアントを作成
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/your_key.json"

from google.cloud import bigquery
client = bigquery.Client()

In [ ]:
# ✅ BigQuery クライアント作成＋検索＋cosine類似度計算
from google.cloud import bigquery
import numpy as np
import ast

client = bigquery.Client(project=os.environ['BQ_PROJECT_ID'])
full_table = f"{os.environ['BQ_PROJECT_ID']}.{os.environ['BQ_DATASET_ID']}.{os.environ['BQ_TABLE_NAME']}"
query = f"SELECT question, answer, embedding FROM `{full_table}`"
rows = client.query(query).result()

user_query = "パスワードを変更したい"
user_embedding = np.random.rand(384)

def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return float(np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2)))

results = []
for row in rows:
    emb = row['embedding']
    if isinstance(emb, str):
        emb = ast.literal_eval(emb)
    score = cosine_similarity(user_embedding, emb)
    results.append((score, row['question'], row['answer']))

results.sort(reverse=True)
top_3 = results[:3]

In [ ]:
# ✅ プロンプト生成
prompt = "あなたはFAQに詳しい市役所のAI案内係です。\n以下は過去の類似質問とその回答です。参考にして、できるだけ簡潔に質問に答えてください。\n\n<FAQ>\n"
for i, (_, q, a) in enumerate(top_3, 1):
    prompt += f"Q{i}: {q}\nA{i}: {a}\n\n"
prompt += "</FAQ>\n\nQ: " + user_query + "\nA:"
print(prompt)

In [ ]:
# ✅ 回答生成
answer = call_chatgpt(prompt)
print("\n===== 回答 =====\n", answer)